## Simple Streamlit App for Evaluation LLM

Simple example showing one way to capture user feedback from using a LLM. Lots of ways to continue to improve and modify this.  

A much better example can be found here: https://llm-eval-demo.streamlit.app/

![Image](Streamlit_Eval.gif)

Saving data to cortex_eval_table

In [ ]:
# Import python packages
import streamlit as st
from datetime import datetime
from streamlit import cache_resource, cache_data
from snowflake.snowpark.session import Session
from snowflake.snowpark.context import get_active_session
import re
import json

session = get_active_session()

@cache_data(show_spinner='CALLING LLM')
def complete(model, prompt):
    global session
    prompt = re.sub("'", "''", prompt)
    sql_code = f"SELECT SNOWFLAKE.CORTEX.COMPLETE('{model}', '{prompt}') as answer"
    df = session.sql(sql_code).to_pandas()
    return df['ANSWER'][0], sql_code


def update_feedback(session, record_id, feedback):
    update_query = f"""
    UPDATE cortex_eval_table
    SET FEEDBACK = ?
    WHERE id = ?
    """
    session.sql(update_query, [feedback, record_id]).collect()

st.header("Evaluation App with Cortex")
st.write("All prompts, responses, and feedback are saved.")
completion_input = st.text_area("Prompt",height=200)
completion_model = st.selectbox("Choose Model", options = ['mixtral-8x7b','mistral-7b'])

if 'record_id' not in st.session_state:
    st.session_state.record_id = None
if 'feedback' not in st.session_state:
    st.session_state.feedback = None

# Function to insert data and retrieve ID
def insert_data_and_get_id(session, prompt, response, model):
    formatted_timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    #Update table
    insert_query = f"""
    INSERT INTO cortex_eval_table (PROMPT, RESPONSE, MODEL, TIMESTAMP, FEEDBACK)
    VALUES (?, ?, ?, ?, ?)
    """
    session.sql(insert_query, [prompt, response, model, formatted_timestamp, None]).collect()

    ##Get ID number
    result = session.sql("SELECT MAX(id) FROM cortex_eval_table").collect()
    new_id = result[0][0]  # Assuming MAX(id) gives us the newest ID safely
    return new_id

# Button to complete the task and save data
if st.button("Complete"):
    if completion_input and completion_model:
        result, sql_code = complete(completion_model, completion_input)
        st.success(result)
        record_id = insert_data_and_get_id(session, completion_input, result, completion_model)
        st.session_state.record_id = record_id
        st.success("Data saved successfully!")
        with st.expander("See SQL"):
            st.code(sql_code, language='SQL')

# Feedback buttons
col1, col2 = st.columns(2)
if col1.button('👍', key='like'):
    st.session_state.feedback = 'like'
    update_feedback(session, st.session_state.record_id, st.session_state.feedback)
    st.success("Thank you for your feedback!", icon="✅")

if col2.button('👎', key='dislike'):
    st.session_state.feedback = 'dislike'
    update_feedback(session, st.session_state.record_id, st.session_state.feedback)
    st.error("Thank you for your feedback!", icon="✅")

# Display message on what feedback was registered
if st.session_state.feedback is not None:
    st.write(f"Feedback {st.session_state.feedback} recorded at ID {st.session_state.record_id}")     
def fetch_data():
    query = "SELECT * FROM cortex_eval_table"
    return session.sql(query).to_pandas()

# Display the results below
st.write("## All Results")
results_df = fetch_data()
st.dataframe(results_df)
